# Track files, in-memory objects & folders [S3 storage]

In [ ]:
!lamin load testuser1/{instance_name}
!lamin delete {instance_name}
!lamin init --storage s3://lamindb-ci --name {instance_name}

In [ ]:
import lamindb as ln
import pytest

## Local files

Some test data.

In [ ]:
pbmc68k = ln.dev.datasets.anndata_pbmc68k_reduced()

Subset to a mini file to speed up the run time of this notebook:

In [ ]:
pbmc68k = pbmc68k[:5, :5].copy()

In [ ]:
pbmc68k

###  Upload from memory using explicit semantic `key`

#### Upload h5ad

In [ ]:
pbmc68k_h5ad = ln.File(pbmc68k, key=f"test-upload/pbmc68k.h5ad")

In [ ]:
pbmc68k_h5ad = ln.save(pbmc68k_h5ad)

In [ ]:
ln.delete(pbmc68k_h5ad, delete_data_from_storage=True)

#### Upload zarr

In [ ]:
# Runs too long, should be tested elsewhere
# pbmc68k_zarr = ln.File(pbmc68k, key="test-upload/pbmc68k.zarr", format="zarr")
# pbmc68k_zarr = ln.save(pbmc68k_zarr)
# ln.delete(pbmc68k_zarr, delete_data_from_storage=True)

### Upload using `id` with implicit `key`

#### Upload h5ad

In [ ]:
pbmc68k_h5ad = ln.File(pbmc68k, name="pbmc68k.h5ad")

In [ ]:
pbmc68k_h5ad = ln.save(pbmc68k_h5ad)

In [ ]:
ln.delete(pbmc68k_h5ad, delete_data_from_storage=True)

#### Upload zarr

In [ ]:
# Runs too long, should be tested elsewhere
# pbmc68k_zarr = ln.File(pbmc68k, name="pbmc68k.zarr", format="zarr")
# pbmc68k_zarr = ln.save(pbmc68k_zarr)
# ln.delete(pbmc68k_zarr, delete_data_from_storage=True)

### Error behaviors

Specified bucket does not exist.

In [ ]:
with pytest.raises(FileNotFoundError):
    pbmc68k_h5ad = ln.File("s3://inexistent-bucket-239809834/pbmc68k.h5ad")

In [ ]:
with pytest.raises(FileNotFoundError):
    pbmc68k_h5ad = ln.File("s3://lndb-setup-ci/pbmc68k.h5ad")

Cross bucket moving of files.

In [ ]:
with pytest.raises(ValueError):
    pbmc68k_h5ad = ln.File("s3://bionty-assets/Species.csv")

## Track existing files

Now we'd like to ingest a csv file that is located in this bucket:

In [ ]:
file = ln.File("s3://lamindb-ci/test-data/test.csv")

In [ ]:
ln.save(file)

Query and load data:

In [ ]:
file = ln.select(ln.File, id=file.id).one()

file

In [ ]:
file.path()

In [ ]:
file.stage()

In [ ]:
file.load().head()

In [ ]:
# Delete the file record
ln.delete(file, delete_data_from_storage=False)

In [ ]:
!lamin delete {instance_name}